In [2]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
X_train = pd.read_csv('input/train.csv', header=None, prefix='col')
X_test = pd.read_csv('input/test.csv', header=None, prefix='col') 
y_train = pd.read_csv('input/train-target.csv', header=None, names=['target']) 
subm = pd.read_csv('input/sample-submission.csv', header=None) 

df = pd.concat([X_train, y_train], axis=1)

# Preproc

In [34]:
drop_cols = ['col17', 'col22', 'col26', 'col9', 'col15', 'col16']
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

In [208]:
from sklearn.decomposition import PCA
pca = PCA(n_components=11, random_state=12)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [209]:
# oof_lgb, prediction_lgb, scores = train_model(X_train_pca, X_test_pca, y, params=params, 
#                                               folds=folds, model_type='lgb', 
#                                               plot_feature_importance=False)

In [ ]:
# CV mean score: 0.8323, std: 0.0106. n=11

X = np.concatenate([X_train.values, X_train_pca], axis=1)
y = y_train.values.flatten()
X_test_np = np.concatenate([X_test.values, X_test_pca], axis=1)

oof_lgb, prediction_lgb, scores = train_model(X, X_test_np, y, params=params, 
                                              folds=folds, model_type='lgb', 
                                              plot_feature_importance=False)

Fold 0 started at Mon Aug  3 00:39:33 2020
Fold 1 started at Mon Aug  3 00:40:00 2020
Fold 2 started at Mon Aug  3 00:40:39 2020
Fold 3 started at Mon Aug  3 00:41:42 2020
Fold 4 started at Mon Aug  3 00:41:56 2020


In [186]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

X_train_pca = pf.fit_transform(X_train)
X_test_pca = pf.transform(X_test)

In [189]:
params_pf = {'boost': 'gbdt',
          'learning_rate': 0.03,
          'max_depth': -1,  
          'metric':'auc',
          'min_data_in_leaf': 21,
          'num_leaves': 251,
          'num_threads': -1,
          'verbosity': -1,
          'objective': 'binary',
         }

In [190]:
X = np.concatenate([X_train.values, X_train_pca], axis=1)
y = y_train.values.flatten()
X_test_np = np.concatenate([X_test.values, X_test_pca], axis=1)

oof_lgb, prediction_lgb, scores = train_model(X, X_test_np, y, params=params_pf, 
                                              folds=folds, model_type='lgb', 
                                              plot_feature_importance=False)

Fold 0 started at Mon Aug  3 00:09:47 2020
Fold 1 started at Mon Aug  3 00:11:09 2020
Fold 2 started at Mon Aug  3 00:13:56 2020
Fold 3 started at Mon Aug  3 00:16:30 2020
Fold 4 started at Mon Aug  3 00:19:46 2020
CV mean score: 0.8258, std: 0.0096.


In [198]:
X_train_round = ((X_train / X_train.max(axis=0)) * 10).round()
X_test_round = ((X_test / X_test.max(axis=0)) * 10).round()

In [199]:
X = np.concatenate([X_train.values, X_train_round], axis=1)
y = y_train.values.flatten()
X_test_np = np.concatenate([X_test.values, X_test_round], axis=1)

oof_lgb, prediction_lgb, scores = train_model(X, X_test_np, y, params=params_pf, 
                                              folds=folds, model_type='lgb', 
                                              plot_feature_importance=False)

Fold 0 started at Mon Aug  3 00:30:40 2020
Fold 1 started at Mon Aug  3 00:31:06 2020
Fold 2 started at Mon Aug  3 00:31:17 2020
Fold 3 started at Mon Aug  3 00:31:33 2020
Fold 4 started at Mon Aug  3 00:31:58 2020
CV mean score: 0.8258, std: 0.0111.


# Model

In [141]:
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

import time

In [170]:
def train_model(X, X_test, y, params, folds, model_type='lgb', plot_feature_importance=False, 
                averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train, params={'verbose': -1})
            valid_data = lgb.Dataset(X_valid, label=y_valid, params={'verbose': -1})
            
            model = lgb.train(params,
                    train_data,
                    num_boost_round=20000,
                    valid_sets = [train_data, valid_data],
                    verbose_eval=False,
                    early_stopping_rounds = 200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = [f'col{ncol}' for ncol in range(X.shape[-1])]
            fold_importance["importance"] = model.feature_importance()
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                   by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction, scores
    
    else:
        return oof, prediction, scores

In [157]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
# folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
# folds = TimeSeriesSplit(n_splits=n_fold)

In [173]:
params = {'boost': 'gbdt',
#           'feature_fraction': 0.5,
          'learning_rate': 0.01,
          'max_depth': -1,  
          'metric':'auc',
          'min_data_in_leaf': 21,
          'num_leaves': 251,
          'num_threads': -1,
          'verbosity': -1,
          'objective': 'binary',
         }

In [159]:
# CV mean score: 0.8263, std: 0.0107.

X = X_train.values
y = y_train.values.flatten()
X_test_np = X_test.values

oof_lgb, prediction_lgb, scores = train_model(X, X_test_np, y, params=params, 
                                              folds=folds, model_type='lgb', 
                                              plot_feature_importance=False)

Fold 0 started at Sun Aug  2 23:37:55 2020
Training until validation scores don't improve for 200 rounds
[1000]	training's auc: 1	valid_1's auc: 0.823243
[2000]	training's auc: 1	valid_1's auc: 0.825826
Early stopping, best iteration is:
[1982]	training's auc: 1	valid_1's auc: 0.825934
Fold 1 started at Sun Aug  2 23:38:37 2020
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[688]	training's auc: 1	valid_1's auc: 0.826597
Fold 2 started at Sun Aug  2 23:38:53 2020
Training until validation scores don't improve for 200 rounds
[1000]	training's auc: 1	valid_1's auc: 0.842033
Early stopping, best iteration is:
[1144]	training's auc: 1	valid_1's auc: 0.842717
Fold 3 started at Sun Aug  2 23:39:18 2020
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[458]	training's auc: 1	valid_1's auc: 0.808969
Fold 4 started at Sun Aug  2 23:39:30 2020
Training until validation scores don't improve for 200 r

In [204]:
roc_auc_score(y, oof_lgb)

0.8306480206200654

In [205]:
prediction_lgb

array([0.04249949, 0.28468248, 0.02818954, ..., 0.03819107, 0.04477126,
       0.9581301 ])

In [206]:
subm.iloc[:, 0] = prediction_lgb

In [207]:
subm.to_csv('input/subm007.csv', header=None, index=None)